In [27]:
import urllib.request
from bs4 import BeautifulSoup
import os
import re

# Функция преобразовывает буквенный месяц
# (дата на сайте указана в формате ДД МЕСЯЦ (ГГГГ)) в числовой

def monthToDigit(month):
    if month == 'января':
        monthDigit = '01'

    elif month == 'февраля':
        monthDigit = '02'

    elif month == 'марта':
        monthDigit = '03'

    elif month == 'апреля':
        monthDigit = '04'

    elif month == 'мая':
        monthDigit = '05'

    elif month == 'июня':
        monthDigit = '06'

    elif month == 'июля':
        monthDigit = '07'

    elif month == 'августа':
        monthDigit = '08'

    elif month == 'сентября':
        monthDigit = '09'

    elif month == 'октября':
        monthDigit = '10'

    elif month == 'ноября':
        monthDigit = '11'

    elif month == 'декабря':
        monthDigit = '12'

    return monthDigit


In [28]:
# Функция поиска (мета)данных по тэгам


def meta(pageUrl):
    try:
        d = {}
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
        soup = BeautifulSoup(text, 'html.parser')
        # поиск заголовка
        d['header'] = soup.find('div', {'class': 'f26'}).get_text()
        # поиск текста статьи
        tagText = soup.find('div', {'id': 'material_text'}).get_text()
        # чистим статью от тэгов html, которые попали в div
        woN = re.sub(r'\n', '', tagText)
        woR = re.sub(r'\r', '', woN)
        woT = re.sub(r'\t', '', woR)
        # чистый текст
        d['text'] = woT.strip()
        # здесь содержится информация об авторе и дате
        footer = soup.find('div', {'id': 'vnizu_a_n_v'})
        children = footer.findChildren('div')
        # строка с именем автора
        authorLine = str(children[0])
        # строка с датой
        dateLine = str(children[1])
        # поиск автора
        author = re.search(
            'Автор: (.*?)(((\.)|(\,))(.*?))?</div>', authorLine)
        linkAuthor = re.search(
            '<a.+?></a>', authorLine)   
        if author and not linkAuthor:
            d['author'] = author.group(1)
        else:
            d['author'] = 'None'
        # поиск даты
        date = re.search('Время: (.*?)</div>', dateLine)
        if date:
            # выделяем год
            year = re.search('[а-я]*? (....) г', date.group(1))
            if year:
                d['year'] = year.group(1)
            # но иногда его нет...
            else:
                d['year'] = 'None'
            # выделяем месяц
            month = re.search('.. ([а-я]*)[ ]*?(.... г)?', date.group(1))
            if month:
                digitMonth = month.group(1)
                d['month'] = monthToDigit(digitMonth.strip())
            # выделяем день
            day = re.search('(..) [а-я]*[ ]*?(.... г)?', date.group(1))
            if day:
                d['day'] = day.group(1)

            return d
    # на случай, если страница сломана
    except:
        print('Error at', pageUrl)


In [29]:
# собственно краулер


def crowler():
    # ссылка на странички
    commonUrl = 'http://www.vecherniy.com/wall?id='
    # огромный словарь всех статей с сайта
    # ключ - номер статьи на сайте,
    # значения - словарь с мета-данными каждой статьи
    bigD = {}
    # считаем статьи
#     изменила количество страниц
    for i in range(1, 1102):
        pageUrl = commonUrl + str(i)
        d = meta(pageUrl)
        if d:
            # если все прошло успешно, и ссылка не сломана -
            # добавляем в мета-данные адрес
            d['url'] = pageUrl
            bigD[i] = d
        # если нет - ничего не делаем
        else:
            pass
    return bigD


In [30]:
# Функция представления даты в формате ДД.ММ.ГГГГ


def date_dots(dictionary):
    goodDate = dictionary['day'] + '.' + dictionary['month'] + '.' + dictionary['year']
    return goodDate


In [31]:
# Функция разбиения текста на предложения и на строки



def break_maker(text):
    dots = re.sub('\. ', '.\\n', text)
    exclamations = re.sub('\! ', '!\\n', dots)
    questions = re.sub('\? ', '?\\n', exclamations)
    return questions


In [32]:
# Функция подсчета статей в папке год\месяц


def enum_art(bigD):
    y = bigD[1]['year']
    m = bigD[1]['month']
    i = 0
    for key, value in bigD.items():
        if bigD[key]['year'] == y and bigD[key]['month'] == m:
            i += 1
            bigD[key]['ordinal'] = i
        else:
            y = bigD[key]['year']
            m = bigD[key]['month']
            i = 1
            bigD[key]['ordinal'] = i
    return bigD


In [33]:
# Функция создает файлы mystem и папки для них
# root - корень папки, т.к. у нас 2 папки с mystem-файлами,
# bigD - большой словарь со всеми статьями,
# abbr - тип файла (.txt или .xml),
# special - пометка для .xml-файлов



def mystemization(root, bigD, abbr, special):
    # местоположение mystem
#     изменила путь к mystem
    way = r'C:\Users\annap\mystem.exe'
    # функции mystem
    settings = ' -cdli --eng-gr '
    for key, values in bigD.items():
        mystemPath = root
        # создаем папку с годом, месяцем, туда записываем статьи
        dirPYMPath = os.path.join(root, bigD[key]['year'], bigD[key]['month'])
        # для первой папки год\месяц
        if not os.path.exists(dirPYMPath):
            os.makedirs(dirPYMPath)
        # каждое предложение с новой строчки
        text = break_maker(bigD[key]['text'])
        # временный файл txt, откуда мы будем брать текст для разметки
        filename = 'input' + str(bigD[key]['ordinal']) + '.txt'
        filepath = os.path.join(dirPYMPath, filename)
        # создаем новый файл и записываем туда текст статьи
        with open(filepath, 'x', encoding='utf-8') as f:
            f.write(text)
        # майстемизация в деле:
        # временный файл
        inp = filepath
        # перманентный файл: articleN.xml
        outfilename = 'article' + str(bigD[key]['ordinal']) + abbr
        # путь к файлу в папке mystem
        out = os.path.join(dirPYMPath, outfilename)
        os.system(way + settings + special + inp + ' ' + out)
        # удаляем временный файл
        os.remove(filepath)


In [34]:
# запись файлa plain-text


def write_article(d, dirpath):
    # файл articleN.txt
    filename = 'article' + str(d['ordinal']) + '.txt'
    # путь к файлу в папке
    filepath = os.path.join(dirpath, filename)
# добавляем путь к неразмеченному тексту - мета-данные
    d['path'] = filepath
    with open(d['path'], 'x', encoding='utf-8') as f:
        # сначала мета-информация...
        f.write('@au {}'.format(d['author']))
        f.write('\n')
        f.write('@ti {}'.format(d['header']))
        f.write('\n')
        f.write('@da {}'.format(date_dots(d)))
        f.write('\n')
        f.write('@topic {}'.format('None'))
        f.write('\n')
        f.write('@url {}'.format(d['url']))
        f.write('\n')
        # ...потом текст
        f.write(d['text'])

    return d


In [35]:
# Функция создает папки для неразмеченного текста


def plain_folders(bigD):
    # у нас есть метаданные: создаем папку с годом, месяцем
    # записываем туда статьи
    for key, values in bigD.items():
        # путь до корневой папки
        plainPath = r'YV\plain'
        dirPYMPath = os.path.join(
            plainPath, bigD[key]['year'], bigD[key]['month'])
        if not os.path.exists(dirPYMPath):
            os.makedirs(dirPYMPath)
        # запись в файл
        bigD[key] = write_article(bigD[key], dirPYMPath)

    return bigD


In [36]:
# Функция создает файл для метаданных в формате csv


def meta_csv(bigD):
    filepath = r'YV\metadata.csv'
    # записываем в файл
    with open(filepath, "x", encoding="utf-8") as f:
        for key, values in bigD.items():
            f.write(
                '{}\t{}\t{}\t{}\tпублицистика\tNone\tнейтральный\tн-возраст\tн-уровень\tгородская\t{}\t"Якутск вечерний"\t{}\tгазета\tРоссия\tРеспублика Саха (Якутия)\tru'
                .format(bigD[key]['path'], bigD[key]['author'],
                bigD[key]['header'], date_dots(bigD[key]),
                bigD[key]['url'], bigD[key]['year']))
            f.write('\n')


In [37]:
def main():
    a = enum_art(crowler())
    meta_csv(plain_folders(a))
    mystemization(
        r'YV\mystem-xml', a, '.xml', '--format xml ')
    mystemization(r'YV\mystem-plain', a, '.txt', '')

if __name__ == '__main__':
    main()


Error at http://www.vecherniy.com/wall?id=6
Error at http://www.vecherniy.com/wall?id=8
Error at http://www.vecherniy.com/wall?id=9
Error at http://www.vecherniy.com/wall?id=11
Error at http://www.vecherniy.com/wall?id=12
Error at http://www.vecherniy.com/wall?id=16
Error at http://www.vecherniy.com/wall?id=17
Error at http://www.vecherniy.com/wall?id=20
Error at http://www.vecherniy.com/wall?id=47
Error at http://www.vecherniy.com/wall?id=48
Error at http://www.vecherniy.com/wall?id=51
Error at http://www.vecherniy.com/wall?id=55
Error at http://www.vecherniy.com/wall?id=56
Error at http://www.vecherniy.com/wall?id=57
Error at http://www.vecherniy.com/wall?id=58
Error at http://www.vecherniy.com/wall?id=59
Error at http://www.vecherniy.com/wall?id=60
Error at http://www.vecherniy.com/wall?id=61
Error at http://www.vecherniy.com/wall?id=62
Error at http://www.vecherniy.com/wall?id=63
Error at http://www.vecherniy.com/wall?id=64
Error at http://www.vecherniy.com/wall?id=66
Error at http